# Super Sample Covariance from `PySSC`
D. Sciotti, Jan. 2022

In this notebook I'll outline the analytical recipe and numerical tools used to compute the SSC for Euclid's photometric probes (both used in isolation and combined in the 3x2pt analysis). The survey specifications and fiducial parameters used are the same as in the [IST:F paper](https://arxiv.org/abs/1910.09273).

### Introduction

Super-sample covariance is a form of cosmic variance arising from the coupling of density modes with wavelenght larger than the linear survey size $L$ ("super-survey" modes) with modes with wavelenght shorter than $L$. This coupling is caused by non-linear evolution, which mixes modes with different wavelength.

In the qualitative sketch below, the red sine represents one of these super-survey, long wavelentgh modes. A mode cannot be sampled if $\lambda=2\pi/k > L$. The sub-survey modes are modulated by the above-mentioned non-linear coupling, making the background density $\delta_b$ measured within the survey area different from the true background density. 

![title](visualize_SSC_spyder_ppt_MAIN.png)

The rigorous analytical expression of the SSC is, for observables projected along the line-of-sight [2]: 

$${\rm Cov_{SSC}} (O_1,O_2) = \iint
{\rm d} V_1 {\rm d} V_2 \frac{\partial o_1}{\partial \delta_b} (z_1) \frac{\partial o_2}{\partial \delta_b} (z_2) \sigma^2(z_1,z_2)$$

with
* ${\rm d} V = r^2(z)\frac{{\rm d} r}{{\rm d} z} {\rm d} z$ the comoving volume element per steradian
* $r(z)$ the comoving distance
* $o_i : O_i = \int {\rm d} V_i o_i$ the observable's comoving density 
* $\frac{\partial o_i}{\partial \delta_b} (z_i)$ the *probe response*, which accounts for the variation of the observable with respect to a change in background density perturbation $\delta_b(z)$.
* $\sigma^2(z_{1}, z_{2}) = \frac{1}{2 \pi^{2}} \int k^{2} \mathrm{~d} k P_m\left(k , z_{12}\right) j_{0}\left(k r_{1}\right) j_{0}\left(k r_{2}\right)$ covariance of background density due to super survey modes modulations.

The SSC is then mainly made up by two main terms: 
* The probe response to a change in bg density, $\frac{\partial o_i}{\partial \delta_b} (z_i)$
* The covariance of the background density due to super survey modes modulations, $\sigma^2(z_1,z_2)$

The probe response is difficult to compute; two possible approaches are to calibrate it from simulations [4] or to use the halo model formalism [5]. In our case, we approximate the probe response to be constant ($=4$) both in redshift and in multipole $\ell$. This assumption is introduced in [1] and tested for GCph; it recovers the full SSC computation with a $<10\%$ discrepancy on the unmarginalized Fisher uncertainties up to $\ell_{max} = 1000$.\
The constant response approximation is currently being tested for WL, for which the analysis is pushed up to $\ell_{max} = 5000$ in the optimistic case.

The $\sigma^2(z_1,z_2)$ term is given by
\begin{equation}
\sigma^{2}\left(z_{1}, z_{2}\right)=\frac{1}{2 \pi^{2}} \int k^{2} \mathrm{~d} k P_{\mathrm{m}}\left(k \mid z_{12}\right) j_{0}\left(k r_{1}\right) j_{0}\left(k r_{2}\right)
\tag{1}
\end{equation}
Note that the matter PS appearing in this integral is the _linear_ $P_{\rm m}(k)$, because the super-survey modes are well described by linear theory.

### Approximating the SSC

Having defined the terms in the full SSC expression, we can employ the "slow response" approximation to simplify it: if we assume the response terms $\frac{\partial o_i}{\partial \delta_b} (z_i)$ to vary slowly with redshift with respect to $\sigma^2(z_{1}, z_{2})$, we can pull them out of the integral and arrive at the expression used in our computations: working with 2D projected observables, of the form 

\begin{equation}
C^{AB}_{ij}(\ell) = \int {\rm d} V W^A_i(z) W^B_j(z) P_{AB}(k_\ell , z)
\label{eq:Cl} \tag{2}
\end{equation}

(using Limber approximation), we reduce the full expression to the approximate form:

$$    {\rm Cov_{SSC}}[C^{AB}_{ij}(\ell),C^{CD}_{kl}(\ell ')] \simeq {\cal R}^{AB}C^{AB}_{ij}(\ell){\cal R}^{CD}C^{CD}_{kl}(\ell ')S^{A,B;C,D}_{i,j;k,l}  $$

with 

$$    S^{A,B;C,D}_{i,j;k,l} = \iint {\rm d} V_1 {\rm d} V_2 \dfrac{W^A_i(z_1)W^B_j(z_1)}{I^{AB}_{ij}}
    \dfrac{W^C_k(z_2)W^D_l(z_2)}{I^{CD}_{kl}}\sigma^2(z_1, z_2)$$

The dimensionless, volume-averaged covariance of the background density contrast, $I^{AB}_{ij} = \int {\rm d} V W^A_i(z) W^B_j(z)$ a normalization factor, and ${\cal R}^{AB}$ the (constant, $=4$) response factor, defined as

$$    \frac{\partial P_{AB}(k_\ell , z_1)}{\partial \delta_b} = 
    {\cal R}^{AB}(k_\ell, z_1) P_{AB}(k_\ell , z_1).$$
    
A more detailed derivation of the approximation can be found in Sect. 2 of [3].

### PySSC

The $S^{A,B;C,D}_{i,j;k,l}$ matrix is the output of the Python module `PySSC`. This is then combined with the response terms and the angular power spectra to produce the Super Sample Covariance term, as outlined above.

The `PySSC` routine ([GitHub](https://github.com/fabienlacasa/PySSC), [readthedocs](https://pyssc.readthedocs.io/en/latest/)) needs as inputs:
* the kernels $W^A_i(z)$ used to compute the angular power spectra (eq. $(2)$)
* the redshift array on which such kernels are computed
* a dictionary of cosmological parameters, passed to `classy` to compute the _linear_ matter PS in eq. $(1)$

In alternative, an object with the same keys as the CLASS `cosmo` objects can be passed instead of the dictionary of parameters (see [here](https://pyssc.readthedocs.io/en/latest/examples.html?highlight=cosmo#Advanced:-use-your-own-cosmology)).

**Note**: the window functions $W^A_i(z)$ (these ones need to be computed on a dense $z$ grid, because of the way `PySSC` implements its integrations). The code works for an arbitrary $\ell$ and $z$ binnings. I can either 


**Note**: We account for the fact that we perform partial sky observations by rescaling the full-sky SSC (above formulas) by the sky fraction.\
To be more accurate, one should introduce the PS of the mask in the analysis. This has been done in [3] and found to have a $\sim 10\%$ difference on the unmarginalised errors w.r.t the approximation just described in the case of large survey areas (such as Euclid's)

### Possible implementation
My code computes the Gaussian-only and the Gauss + SSC covariance in harmonic space. 

The necessary ingredients for the Gaussian covariance are:
* the $C^{AB}_{ij}(\ell)$ 
* the noise spectra, $N^{AB}_{ij}(\ell)$
* $\ell$ and $\Delta \ell$ values, or a prescription to compute them (e.g. 30 logarithmically equispaced bins) 

The necessary ingredients for the SS covariance are listed in the cell above (they are the same as the inputs needed by PySSC). \
Another option is to produce independently the $C^{AB}_{ij}(\ell)$ and $W^A_i(z)$ (using Marco Bonici's `Julia` code [`CosmoCentral`](https://github.com/marcobonici/CosmoCentral.jl), which implements IST:F recipe); in this case a fiducial cosmology would be needed.



### References
This is the main reference. It presents the "slow response" approximation and introduces the code used to compute the SSC. It also present a discussion of the effect's impact in the case of a Euclid-like GCph survey (using only one redshift bin):
* [1] Lacasa, F. & Grain, J. 2019, Astronomy  Astrophysics, 624, A61
* [2] Lacasa,  F.  &  Rosenfeld,  R.  2016,  Journal  of  Cosmology  and  AstroparticlePhysics, 2016, 005–005
* [3] Gouyou Beauchamps, S.,Lacasa, F., Tutusaus, I., et al. 2021 [arXiv:2109.02308]
* [4] Barreira, A., Krause, E., & Schmidt, F. 2018b, Journal of Cosmology and Astroparticle Physics, 2018, 015–015
* [5] Rizzato, M., Benabed, K., Bernardeau, F., Lacasa, F. 2019, Monthly Notices of the Royal Astronomical Society, 490, 4688–4714
